# MVP - Engenharia de Dados
## Autor: Marcel Antonio Lopes Pereira

## Objetivo:

Este MVP tem como objetivo construir um pipeline de dados utilizando tecnologias em nuvem para buscar, coletar, modelar, carregar e analisar dados históricos do futebol de seleções masculinas. O propósito central é responder a uma questão amplamente debatida entre torcedores e especialistas:

**Existe alguma seleção nacional masculina dominante na história do futebol a ponto de ser considerada a melhor de todos os tempos?**

Para explorar essa questão, o pipeline permitirá a análise de algumas perguntas-chave, que ajudarão a identificar as seleções que se destacaram ao longo da história:

1. **Qual seleção tem o maior número de vitórias em jogos oficiais e amistosos?**  
   Identificar as seleções com o maior número de vitórias em diferentes tipos de jogos, fornecendo uma base para medir a consistência no desempenho.

2. **Qual seleção tem a melhor taxa de aproveitamento (pontos ganhos por jogo)?**  
   Avaliar a eficiência das seleções, levando em consideração o número de pontos conquistados em relação aos jogos disputados, ajudando a medir o desempenho em termos de consistência.

3. **Qual seleção tem o maior saldo de gols (gols marcados - gols sofridos) em toda a história?**  
   Analisar o saldo de gols das seleções, refletindo sua capacidade ofensiva e defensiva ao longo do tempo.

4. **Qual seleção tem o maior número de vitórias contra outras grandes seleções (top 10 do ranking da FIFA)?**  
   Analisar o desempenho das seleções contra as mais fortes equipes do mundo, refletindo sua competitividade no cenário global.

5. **Qual seleção tem a maior sequência invicta em toda a história?**  
   Identificar quais seleções mantiveram longas sequências sem derrotas, um bom indicativo de consistência e resistência ao longo do tempo.

6. **Qual seleção tem o maior número de jogos sem sofrer gols em toda a história?**  
   Analisar a capacidade defensiva das seleções, refletindo uma defesa sólida e organização tática eficaz ao longo dos anos.

7. **Quais seleções tiveram os artilheiros mais consistentes e como o desempenho desses jogadores está relacionado com o sucesso da equipe?**  
   Relacionar o desempenho individual dos artilheiros com o desempenho da seleção, ajudando a identificar como a contribuição dos goleadores impactou as vitórias e o sucesso da equipe ao longo da história.



## Estrutura do Pipeline

O pipeline de dados será estruturado em três camadas principais:

- **Bronze:** coleta e armazenamento dos dados brutos conforme os arquivos originais.  
- **Silver:** limpeza, padronização e transformação dos dados brutos, garantindo consistência e qualidade para as análises. Nesta etapa, serão tratados problemas como dados ausentes, duplicidades e formatação inadequada.  
- **Gold:** modelagem analítica dos dados para responder às perguntas de negócio, além da otimização para consultas complexas, garantindo eficiência na extração de insights e facilitando a análise exploratória e preditiva.

A análise final dos dados será realizada em um notebook específico, onde todas as perguntas levantadas serão respondidas. A partir dos resultados obtidos, será possível avaliar, com base em dados, quais seleções se destacam historicamente e se alguma delas pode ser considerada a mais dominante de todos os tempos.





## Busca dos dados:

Na busca dos dados necessários para solução do problema, foi escolhida o Kaggle como fonte para obtenção dos mesmos. Nesse processo, foram escolhidos dois conjuntos de dados que são descritos a seguir.

O primeiro conjunto é referente aos resultados de partidas entre seleções no período entre 1872 e 2025. Nesses dados não estão incluídos os resultados de partidas em Jogos Olímípicos, com seleções nacionais B, sub-23 e seleções com melhores jogadores de ligas. O link de acesso a esse conjunto de dados é o seguinte:
https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017

Esses dados foram publicados pelo usuário Mart Jürisoo e são de domínio público (CC0: Public Domain). O conjunto possui 4 arquivos do tipo csv: results.csv (resultados das partidas); shootouts.csv (disputas de penalties); goalscorers.csv (marcadores de gols); former_names.csv (nomes anteriores das equipes).

O segundo conjunto é referente ao Ranking da FIFA de seleções no período de 1992 a 2024. Estão listadas as posições dos times e suas mudanças ao longo do tempo dentro desse intervalo. O link de acesso a esse conjunto de dados é o seguinte:
https://www.kaggle.com/datasets/cashncarry/fifaworldranking

Esses dados forma publicados pelo usuário Alex e também são de domínio público (CC0: Public Domain). Apesar do conjunto possuir 3 arquivos do tipo csv, somente foi utilizado um deles nesse trabalho: fifa_ranking-2024-06-20.csv (Lista de Rankings até 20-06-24).

## Coleta, modelagem e carga dos dados:

Após identificar os dados necessários para o problema, é realizada a coleta e a carga dos mesmos para o ambiente de nuvem. Foi utilizado nesse processo o sistema de armazenamento do Datrabricks com utilização do framework Spark para carga dos dados.


In [0]:
# Importando as bibliotecas necessárias para carga e manipulação das tabelas
import pandas as pd

### Camada Bronze

Na camada bronze, são importados os dados no seu formato bruto a partir dos arquivos csv de origem. São realizados scripts relativos a criação da database onde serão armazenados as tabelas da camada bronze.

In [0]:
%sql
-- Desfazendo o database bronze para ser "recriado"
DROP DATABASE IF EXISTS bronze CASCADE;

In [0]:
%sql 
-- Criando o database bronze
CREATE DATABASE bronze;

Após a criação do database, os arquivos CSV são carregados utilizando a biblioteca Pandas para gerar DataFrames a partir dos respectivos arquivos. Foi realizado download dos arquivos no Kaggle e os mesmos foram salvos no mesmo repositório onde esse Notebook está alocado. Em seguida, esses DataFrames são convertidos para o formato Spark e armazenados como tabelas no sistema de armazenamento em nuvem do Databricks, dentro da camada Bronze do database criado.

In [0]:
# Dicionário com os nomes das tabelas e URLs dos arquivos
csv_files = {
    "results": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/results.csv",
    "shootouts": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/shootouts.csv",
    "goalscorers": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/goalscorers.csv",
    "former_names": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/former_names.csv",
    "fifa_ranking": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/fifa_ranking-2024-06-20.csv"
}

# Loop para carregar, converter e salvar cada arquivo
for table_name, url in csv_files.items():
    # Carregar o CSV como DataFrame Pandas
    df = pd.read_csv(url)
    
    # Converter para DataFrame Spark
    spark_df = spark.createDataFrame(df)
    
    # Salvar como tabela na camada bronze
    spark_df.write.mode("overwrite").saveAsTable(f"bronze.{table_name}")

    print(f"Tabela bronze.{table_name} carregada com sucesso.")


Tabela bronze.results carregada com sucesso.
Tabela bronze.shootouts carregada com sucesso.
Tabela bronze.goalscorers carregada com sucesso.
Tabela bronze.former_names carregada com sucesso.
Tabela bronze.fifa_ranking carregada com sucesso.


Após a execução dos script acima, a camada bronze foi construída com as tabelas. Em seguida, são realizadas verificações Spark SQL para confirmar a criação das tabelas. 


In [0]:
# Lista de tabelas que foram criadas
tables = ["results", "shootouts", "goalscorers", "former_names", "fifa_ranking"]

# Exibindo as 10 primeiras linhas de cada tabela
for table in tables:
    print(f"🔍 Exibindo 10 primeiras linhas da tabela bronze.{table}")
    display(spark.sql(f"SELECT * FROM bronze.{table} LIMIT 10"))

# Exibindo o esquema de cada tabela
for table in tables:
    print(f"📊 Esquema da tabela bronze.{table}")
    spark.sql(f"DESCRIBE TABLE bronze.{table}").show()


🔍 Exibindo 10 primeiras linhas da tabela bronze.results


date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,false
1873-03-08,England,Scotland,4,2,Friendly,London,England,false
1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,false
1875-03-06,England,Scotland,2,2,Friendly,London,England,false
1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,false
1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,false
1877-03-03,England,Scotland,1,3,Friendly,London,England,false
1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,false
1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,false
1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,false


🔍 Exibindo 10 primeiras linhas da tabela bronze.shootouts


date,home_team,away_team,winner,first_shooter
1967-08-22,India,Taiwan,Taiwan,null
1971-11-14,South Korea,Vietnam Republic,South Korea,null
1972-05-07,South Korea,Iraq,Iraq,null
1972-05-17,Thailand,South Korea,South Korea,null
1972-05-19,Thailand,Cambodia,Thailand,null
1973-04-21,Senegal,Ghana,Ghana,null
1973-06-14,Guinea,Mali,Guinea,null
1973-06-14,Mauritius,Tanzania,Mauritius,null
1973-07-26,Malaysia,Kuwait,Malaysia,null
1973-07-26,Cambodia,Singapore,Singapore,null


🔍 Exibindo 10 primeiras linhas da tabela bronze.goalscorers


...

🔍 Exibindo 10 primeiras linhas da tabela bronze.former_names


current,former,start_date,end_date
Benin,Dahomey,1959-11-08,1975-11-30
Burkina Faso,Upper Volta,1960-04-14,1984-08-04
Curaçao,Netherlands Antilles,1957-03-03,2010-10-10
Czechoslovakia,Bohemia,1903-04-05,1919-01-01
Czechoslovakia,Bohemia and Moravia,1939-01-01,1945-05-01
Czechoslovakia,Representation of Czechs and Slovaks,1993-03-24,1993-11-17
DR Congo,Belgian Congo,1948-05-25,1956-01-02
DR Congo,Congo-Léopoldville,1963-04-12,1964-07-19
DR Congo,Congo-Kinshasa,1965-01-09,1970-11-24
DR Congo,Zaïre,1971-01-10,1997-04-27


🔍 Exibindo 10 primeiras linhas da tabela bronze.fifa_ranking


rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
140.0,Brunei Darussalam,BRU,2.0,0.0,140,AFC,1992-12-31
33.0,Portugal,POR,38.0,0.0,33,UEFA,1992-12-31
32.0,Zambia,ZAM,38.0,0.0,32,CAF,1992-12-31
31.0,Greece,GRE,38.0,0.0,31,UEFA,1992-12-31
30.0,Algeria,ALG,39.0,0.0,30,CAF,1992-12-31
29.0,Yugoslavia,YUG,39.0,0.0,29,UEFA,1992-12-31
28.0,Wales,WAL,40.0,0.0,28,UEFA,1992-12-31
27.0,Côte d'Ivoire,CIV,41.0,0.0,27,CAF,1992-12-31
34.0,Austria,AUT,38.0,0.0,34,UEFA,1992-12-31
26.0,Bulgaria,BUL,41.0,0.0,26,UEFA,1992-12-31


📊 Esquema da tabela bronze.results
+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|      date|   string|   null|
| home_team|   string|   null|
| away_team|   string|   null|
|home_score|   bigint|   null|
|away_score|   bigint|   null|
|tournament|   string|   null|
|      city|   string|   null|
|   country|   string|   null|
|   neutral|  boolean|   null|
+----------+---------+-------+

📊 Esquema da tabela bronze.shootouts
+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|         date|   string|   null|
|    home_team|   string|   null|
|    away_team|   string|   null|
|       winner|   string|   null|
|first_shooter|   string|   null|
+-------------+---------+-------+

📊 Esquema da tabela bronze.goalscorers
+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|     date|   string|   null|
|home_team|   string|   null|
|away_team|   string|   null

Nesta etapa do projeto, foi concluída a ingestão dos dados brutos provenientes de diversos arquivos CSV históricos relacionados às seleções nacionais de futebol, utilizando o Spark no Databricks. Com a criação de tabelas na camada bronze, os dados foram armazenados em sua forma original, preservando todas as informações com a maior fidelidade possível.

Agora que a camada bronze está criada e os dados estão prontos para análise, a próxima etapa do trabalho será a construção da camada silver, onde será feita a limpeza, transformação e integração dos dados para garantir que estejam no formato adequado para análises mais complexas.

A camada silver será responsável por aprimorar a qualidade dos dados, identificando e corrigindo inconsistências, transformando variáveis e aplicando quaisquer outras transformações necessárias para garantir que os dados estejam prontos para a análise e construção de modelos na camada gold.

Com isso, segue-se para a próxima fase do pipeline de dados, garantindo que os dados estejam cada vez mais preparados para responder às questões relacionadas à dominância das seleções nacionais no futebol.